## REPLICA OF OTHER TABLE - CHECK WHAT IS GOING ON IN KANSAS

In [17]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [18]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [19]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [21]:
# read nyc origin cbgs
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')

In [22]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

## RUN FOR RENO COUNTY, KS, 2019

In [23]:
# set date variables, in this case, a test month for 2 different years
y0 = '2019'
y1 = '2020'
monthList =["01","02","03","04","05","06", "07","08","09","10","11","12"]
dayNumList =[31, 28, 31, 30, 31, 30, 31, 31,30,31,30,31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [24]:
## Iterate and create pivot for home county to destination county (in region + outside)
frames = [] 
for m in range (0,9):
    for d in range(0, dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y0}/{monthList[m]}/{dayList[d]}/{y0}-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'{y0}-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')
            
            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #separate home trips from other trips for later aggregation
            #id destinations by county fips
            df['dest_stco'] = df['destination_cbg'].str[:5]
            df['dest_tract'] = df['destination_cbg'].str[:11]
            #RENO KANSAS ONLY
            df = df[df['dest_stco'] == '20155']
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','dest_cbg_count','dest_tract']]
            frames.append(dff) 

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01

In [25]:
df_19 = pd.concat(frames)
df_19 = pd.pivot_table(df_19,values=['dest_cbg_count'],index=['date_y-m-d','orig_stco'],columns=['dest_tract'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [26]:
df_19.head(10)

dest_cbg_count                                      \
dest_tract              20155000100 20155000200 20155000300 20155000500   
date_y-m-d orig_stco                                                      
2019-01-01 36005                  0           0           0           0   
           36047                  0           0           0           0   
           36061                  0           0           0           0   
           36081                  0           0           0           0   
2019-01-02 36005                  0           0           0           0   
           36047                  0           0           0           0   
           36061                  0           0           0           0   
           36081                  0           0           0           0   
           36085                  0           0           0           0   
2019-01-03 36005                  0           0           0           0   

                                                                      \
dest_tract           20155000600 20155000700 20155000800 20155001000   
date_y-m-d orig_stco                                                   
2019-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
2019-01-02 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   
2019-01-03 36005               0           0           0           0   

                                                                      \
dest_tract           20155001100 20155001200 20155001300 20155001400   
date_y-m-d orig_stco                                                   
2019-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
2019-01-02 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   
2019-01-03 36005               0           0           0           0   

                                                                          
dest_tract           20155001500 20155001600 20155001700 20155001800 All  
date_y-m-d orig_stco                                                      
2019-01-01 36005               0           0           0           3   3  
           36047               0           0           0           5   5  
           36061               0           0           0           8   8  
           36081               0           0           0           2   2  
2019-01-02 36005               0           0           0           8   8  
           36047               0           0           0           7   7  
           36061               0           0           0          15  15  
           36081               0           0           0           5   5  
           36085               0           0           0           1   1  
2019-01-03 36005               0           0           0           9   9

In [27]:
#Run for 2020

In [28]:
#Fix for leap year
dayNumList[1] = 29 

In [29]:
## Iterate and create pivot for home county to destination county (in region + outside)
frames = [] 
for m in range (0,9):
    for d in range(0, dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y1}/{monthList[m]}/{dayList[d]}/{y1}-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'{y1}-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')
            
            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #separate home trips from other trips for later aggregation
            #id destinations by county fips
            df['dest_stco'] = df['destination_cbg'].str[:5]
            df['dest_tract'] = df['destination_cbg'].str[:11]
            #RENO KANSAS ONLY
            df = df[df['dest_stco'] == '20155']
            #Make new table with select columns
            dff = df[['date_y-m-d','orig_stco','dest_cbg_count','dest_tract']]
            frames.append(dff) 

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31

In [30]:
df_20 = pd.concat(frames)
df_20 = pd.pivot_table(df_20,values=['dest_cbg_count'],index=['date_y-m-d','orig_stco'],columns=['dest_tract'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [31]:
df_20.head(10)

dest_cbg_count                                      \
dest_tract              20155000100 20155000200 20155000300 20155000400   
date_y-m-d orig_stco                                                      
2020-01-01 36005                  0           0           0           0   
           36047                  0           0           0           0   
           36061                  0           0           0           0   
           36081                  0           0           0           0   
           36085                  0           0           0           0   
2020-01-02 36005                  0           0           0           0   
           36047                  0           0           0           0   
           36061                  0           0           0           0   
           36081                  0           0           0           0   
           36085                  0           0           0           0   

                                                                      \
dest_tract           20155000500 20155000600 20155000700 20155000800   
date_y-m-d orig_stco                                                   
2020-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   
2020-01-02 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   

                                                                      \
dest_tract           20155001000 20155001100 20155001300 20155001400   
date_y-m-d orig_stco                                                   
2020-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   
2020-01-02 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
           36085               0           0           0           0   

                                                                           
dest_tract           20155001500 20155001600 20155001700 20155001800  All  
date_y-m-d orig_stco                                                       
2020-01-01 36005               0           0           0         193  193  
           36047               0           0           0         290  290  
           36061               0           0           0         324  324  
           36081               0           0           0         214  214  
           36085               0           0           0          93   93  
2020-01-02 36005               0           0           0         108  108  
           36047               0           0           0         178  178  
           36061               0           0           0         157  157  
           36081               0           0           0         142  142  
           36085               0           0           0          54   54

In [ ]:
# Run another iteration for just counts of devices
#'device_count','completely_home_device_count','part_time_work_behavior_devices','full_time_work_behavior_devices',

In [ ]:
#Run a concat of all three tables to make master table with origin/day index

In [32]:
df_nycxreno = pd.concat([df_19,df_20])

In [33]:
df_nycxreno.head()

dest_cbg_count                                      \
dest_tract              20155000100 20155000200 20155000300 20155000400   
date_y-m-d orig_stco                                                      
2019-01-01 36005                  0           0           0         NaN   
           36047                  0           0           0         NaN   
           36061                  0           0           0         NaN   
           36081                  0           0           0         NaN   
2019-01-02 36005                  0           0           0         NaN   

                                                                      \
dest_tract           20155000500 20155000600 20155000700 20155000800   
date_y-m-d orig_stco                                                   
2019-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
2019-01-02 36005               0           0           0           0   

                                                                      \
dest_tract           20155001000 20155001100 20155001200 20155001300   
date_y-m-d orig_stco                                                   
2019-01-01 36005               0           0         0.0           0   
           36047               0           0         0.0           0   
           36061               0           0         0.0           0   
           36081               0           0         0.0           0   
2019-01-02 36005               0           0         0.0           0   

                                                                      \
dest_tract           20155001400 20155001500 20155001600 20155001700   
date_y-m-d orig_stco                                                   
2019-01-01 36005               0           0           0           0   
           36047               0           0           0           0   
           36061               0           0           0           0   
           36081               0           0           0           0   
2019-01-02 36005               0           0           0           0   

                                      
dest_tract           20155001800 All  
date_y-m-d orig_stco                  
2019-01-01 36005               3   3  
           36047               5   5  
           36061               8   8  
           36081               2   2  
2019-01-02 36005               8   8

In [ ]:
#df_nycxreno = df_nycxreno.reset_index()

In [34]:
df_nycxreno.to_excel('output/Destinations/Reno_Kansas_QAQC.xlsx')

In [ ]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')